In [22]:
def chinese_word_cut(mytext):
    # jieba.load_userdict('prepare/用户自定义词典.txt');#加入自定义语料库
     words = pseg.cut(mytext)
     line_List=[]
     stopwords = open("C:/Users/张珍珍/project/DTMmodel/prepare/stopwords.txt",'r',encoding='utf-8').read().split('\n')
     #flags=['n','v','a','x','adv','vd','d','b']
     #flag_List=wordFlag.split(",")
     for w in words:
        i=0;
         #加入自定义语料库
        if(w.flag not in ['nt','ns','r','t','m','b','j','s','f','nz','nr','c','ad'] and len(re.findall(r'[\u4e00-\u9fff]+',w.word))>0 and (not w.word.isnumeric()) and w.word not in stopwords and len(re.findall('\s',w.word))==0):
             line_List.append(w.word)
        i=i+1
     line=""
     for word in line_List:
          line+=word+" "
     return line

In [23]:
import numpy,re
import pandas as pd
import jieba.posseg as pseg
corpus=pd.read_csv("D:/电力短缺新闻/电力短缺日报.csv")["内容"].tolist()
texts=[]
stopwords = open("C:/Users/张珍珍/project/DTMmodel/prepare/stopwords.txt",'r',encoding='utf-8').read()
stopwords = stopwords.split('\n')
for corpu in corpus:
    try:
        texts.append(chinese_word_cut(corpu))
    except:
        print(1)

1


In [15]:
n_features = 3000000000
tfidf2 = TfidfVectorizer(token_pattern=r"(?u)\b[\u4e00-\u9fff]{2,}\b",ngram_range=(1,1),
    strip_accents = 'unicode',
    max_features=n_features,
    # stopwords=stopwords,
      max_df = 0.8,
      min_df = 10)
tfidf_matrix= tfidf2.fit_transform(texts)
feature_names=tfidf2.get_feature_names_out()
import pandas as pd
data = {'word': feature_names,
        'tfidf': tfidf_matrix.toarray().mean(axis=0).tolist()}
df = pd.DataFrame(data)
df.sort_values(by="tfidf" , ascending=False).head(100)

,word,tfidf
9186,用电,0.078766
1291,供电,0.063949
1394,保电,0.045568
9328,电网,0.045411
9221,电力,0.041724
...,...,...
8632,火电,0.007919
5221,应对,0.007873
12503,避峰,0.007836
1303,供需,0.007807


In [16]:
df.to_csv("D:/电力短缺新闻/tf-idf1.csv",encoding='utf-8-sig')

In [12]:
import numpy as np
import pandas as pd
import jieba
import jieba.posseg as pseg
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import codecs
import os
import re

In [17]:
from nltk.text import TextCollection
from nltk.tokenize import word_tokenize

#首先，构建语料库corpus
sents=[word_tokenize(sent) for sent in texts] #对每个句子进行分词
#print(sents)  #输出分词后的结果
corpus=TextCollection(sents)  #构建语料库
#print(corpus)  #输出语料库

#计算语料库中"one"的tf值

In [19]:
sents

[[],
 [],
 [],
 ['晨报',
  '国网',
  '电力',
  '进博',
  '前线',
  '指挥',
  '智能',
  '巡检',
  '有条不紊',
  '开关柜',
  '温度',
  '湿度',
  '局部',
  '放电',
  '压板',
  '状态',
  '巡检',
  '输电',
  '通道',
  '可视化',
  '拍',
  '同步',
  '传回',
  '重要',
  '输电',
  '通道',
  '安全',
  '语音',
  '助手',
  '小',
  '电',
  '实时',
  '播报',
  '核心区',
  '电网',
  '负荷',
  '电网',
  '正',
  '最佳',
  '状态',
  '进博',
  '保电',
  '实战',
  '阶段',
  '电力',
  '进',
  '博会',
  '全景',
  '供电',
  '保障系统',
  '版',
  '迭代',
  '升级',
  '系统',
  '将国',
  '电力',
  '套源',
  '端',
  '系统集成',
  '屏',
  '保电',
  '区域',
  '工单',
  '数字化',
  '保电',
  '护航',
  '进',
  '博会',
  '核心',
  '区域',
  '电网',
  '供电',
  '可靠性'],
 ['针对',
  '能源',
  '供需',
  '偏紧',
  '发展',
  '改革',
  '委',
  '经济运行',
  '调节',
  '局',
  '始终',
  '确保',
  '民生',
  '用能',
  '保供',
  '重中之重',
  '确保',
  '安全',
  '温暖',
  '措',
  '并举',
  '加强',
  '供需',
  '调节',
  '确保',
  '能源',
  '稳定',
  '供应',
  '措施',
  '确保',
  '用能',
  '安全',
  '问',
  '限电',
  '引发',
  '社会',
  '能源',
  '保供',
  '担心',
  '发展',
  '改革',
  '委',
  '采取',
  '措施',
  '保障',
  '能源供应',
  '答',
  '重点',
  '确保',
  '能源'

In [16]:
words=[]
for j in texts:
    for i in j.split(' '):
        words.append(i)

In [21]:
contents=[]
import tqdm
for word in tqdm.tqdm(set(words)):
    if(len(word)>=2):
        tf=corpus.tf(word,corpus)    # 1/12
        #计算语料库中"one"的idf值
        idf=corpus.idf(word)      #log(3/1)
        #计算语料库中"one"的tf-idf值
        tf_idf=corpus.tf_idf(word,corpus)
        contents.append([word,tf,idf,tf-idf])

100%|██████████████████████████████████████████████████████████████████████████| 69714/69714 [2:01:06<00:00,  9.59it/s]


In [24]:
import pandas
df=pandas.DataFrame(contents)
df.columns=['A','B','C','D']

In [29]:
df_sorted = df.sort_values(by='D',ascending=False)
print(df_sorted)

         A             B         C         D
56052   用电  2.676805e-02  0.455463 -0.428695
35081   供电  2.019201e-02  0.457521 -0.437329
52250   电力  1.329753e-02  0.711803 -0.698506
41344   电网  1.254815e-02  0.788066 -0.775518
67114   确保  7.446818e-03  0.785714 -0.778267
...    ...           ...       ...       ...
42361   平直  3.878753e-07  9.974831 -9.974831
20660   饰面  3.878753e-07  9.974831 -9.974831
42355   窗饰  3.878753e-07  9.974831 -9.974831
42354   想问  3.878753e-07  9.974831 -9.974831
0      忍受力  3.878753e-07  9.974831 -9.974831

[69713 rows x 4 columns]


In [ ]:
df_sorted.head(50)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from math import log
# 示例文档集合
# 要计算TF和IDF的词语
# 文档预处理
listz=[]
# 计算TF值
for search_word in tqdm.tqdm(wordspre):
    counts=0
    doocs=0
    for doc in preprocessed_documents:
        counts+=doc.count(search_word)
        doocs+=len(doc)
    tf_value=counts/doocs
    # 计算IDF值
    num_documents = len(preprocessed_documents)
    num_documents_with_word = sum([1 for doc in preprocessed_documents if search_word in doc])
    idf_value = log(num_documents / (num_documents_with_word + 1))
    tf_idf=tf_value/idf_value
    listz.append([tf_value,idf_value,tf_idf])

 85%|██████████████████████████████████████████████████████████████▊           | 59121/69714 [1:12:02<14:52, 11.86it/s]

In [46]:
print(counts/doocs)

0.0


In [60]:
wordspre=preprocessed_documents

In [63]:
wordspre=set(i for j in wordspre for i in j)

In [65]:
len(set(wordspre))

69714